In [1]:
import sys
import os
sys.path.append('C:\\Users\\Administrator\\Desktop\\风控产品\\risk_project')
from risk_models.config.read_config.read_func import Read_Oracle
from risk_models.config.write_config.write_func import Write_Oracle
import numpy as np
import datetime

In [2]:
STARTDT='2021-01-01'

In [3]:
sql='''select * from opening_inventory where iscurrent =1 '''
opening_inventory  = Read_Oracle().read_oracle(sql=sql,database='dbods')
sql='''select * from opening_inventory_detail where iscurrent =1'''
opening_inventory_detail  = Read_Oracle().read_oracle(sql=sql,database='dbods')
sql="""select ORG_CODE, QTY, QTY_CO,UNIT_CO,ACTRUAL_STOCK_DATE,TRADE_TOTAL, BILL_TYPE,NET_WT from ods_zmxpq.EMS_STOCK_BILL 
where CAPXACTION != 'D'
and  BILL_TYPE in ('1','2', '3', '4', '5', '6', 'A', 'B') 
and ACTRUAL_STOCK_DATE >= date'{}' """.format(STARTDT)
EMS_STOCK_BILL= Read_Oracle().read_oracle(sql=sql,database='dbods')
df = EMS_STOCK_BILL.reset_index()
def transfer(df,columns_list,dtype):
    for col in columns_list:
        df['{}'.format(col)] = df['{}'.format(col)].astype(dtype)
transfer(df,['QTY_CO','QTY','TRADE_TOTAL','NET_WT'],float)
now = datetime.datetime.strftime(datetime.datetime.now(), format='%Y-%m-%d %H:%M:%S')
df['CHECK_TIME'] = now
df['CHECK_TIME'] = df['CHECK_TIME'].astype('datetime64')
BD_RISK_DETAIL_STOCK_ST3 = df.drop('index',axis=1).reset_index().rename(columns={'index':'ID'})
BD_RISK_DETAIL_STOCK_ST3

Read Table successfully! Total read time spent 0.25902438163757324s
Read Table successfully! Total read time spent 3.4760782718658447s


In [9]:
Write_Oracle().write_oracle('BD_RISK_DETAIL_STOCK_ST3',BD_RISK_DETAIL_STOCK_ST3)

Read Table successfully! Total read time spent 0.0900106430053711s
Processing... Writing  432107  rows into database
Insert data into BD_RISK_DETAIL_STOCK_ST3 successfully! Total write time spent 33.79624366760254s
